In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [3]:
feed.routes.route_type.iat[0] = 2  # Create another route type
trip_stats = feed.compute_trip_stats()

In [4]:
#dates = ["20100101"]
week = feed.get_first_week()
dates = [week[0], week[2]] + ["20100101"]
f = feed.compute_feed_stats(trip_stats, dates=dates, split_route_types=True)
f.T

,0,1,2,3
route_type,2,3,2,3
num_stops,66,386,66,386
num_routes,1,19,1,19
num_trips,59,563,59,563
num_trip_starts,59,563,59,563
num_trip_ends,58,559,58,559
service_distance,1894.22,11879.8,1894.22,11879.8
service_duration,57.35,415.25,57.35,415.25
service_speed,33.0292,28.6088,33.0292,28.6088
peak_num_trips,5,35,5,35


In [108]:
freq = "12H"
sts = feed.compute_stop_time_series(dates, freq=freq, split_directions=True)
rts = feed.compute_route_time_series(trip_stats, dates, freq=freq)
fts = feed.compute_feed_time_series(trip_stats, dates, freq=freq, split_route_types=True)

In [109]:
rts

indicator           num_trip_ends                                           \
route_id                  110-423 111-423 112-423 113-423 120-423 120N-423   
datetime                                                                     
2014-05-26 00:00:00          19.0    18.0     4.0     3.0    11.0      0.0   
2014-05-26 12:00:00          39.0    39.0    11.0     3.0    21.0      2.0   
2014-05-27 00:00:00           NaN     NaN     NaN     NaN     NaN      NaN   
2014-05-27 12:00:00           NaN     NaN     NaN     NaN     NaN      NaN   
2014-05-28 00:00:00          19.0    18.0     4.0     3.0    11.0      0.0   
2014-05-28 12:00:00          39.0    39.0    11.0     3.0    21.0      2.0   

indicator                                            ... service_speed  \
route_id            121-423 122-423 123-423 130-423  ...       131-423   
datetime                                             ...                 
2014-05-26 00:00:00    13.0    13.0    21.0    11.0  ...     24.049902   
2014-05-26 12:00:00    21.0    20.0    38.0    22.0  ...     24.051800   
2014-05-27 00:00:00     NaN     NaN     NaN     NaN  ...           NaN   
2014-05-27 12:00:00     NaN     NaN     NaN     NaN  ...           NaN   
2014-05-28 00:00:00    13.0    13.0    21.0    11.0  ...     24.049902   
2014-05-28 12:00:00    21.0    20.0    38.0    22.0  ...     24.051800   

indicator                                                                   \
route_id              131N-423    133-423    140-423    141-423    142-423   
datetime                                                                     
2014-05-26 00:00:00   0.000000  25.035081  25.634669  20.802088  26.021598   
2014-05-26 12:00:00  24.067374  26.439134  25.832253  20.828188  26.417789   
2014-05-27 00:00:00        NaN        NaN        NaN        NaN        NaN   
2014-05-27 12:00:00        NaN        NaN        NaN        NaN        NaN   
2014-05-28 00:00:00   0.000000  25.035081  25.634669  20.802088  26.021598   
2014-05-28 12:00:00  24.067374  26.439134  25.832253  20.828188  26.417789   

indicator                                                       
route_id               143-423   143W-423    150-423  150E-423  
datetime                                                        
2014-05-26 00:00:00  24.366138   0.000000  31.565566   0.00000  
2014-05-26 12:00:00  24.307749  30.331218  31.547064  34.12138  
2014-05-27 00:00:00        NaN        NaN        NaN       NaN  
2014-05-27 12:00:00        NaN        NaN        NaN       NaN  
2014-05-28 00:00:00  24.366138   0.000000  31.565566   0.00000  
2014-05-28 12:00:00  24.307749  30.331218  31.547064  34.12138  

[6 rows x 120 columns]

In [110]:
fts

indicator           num_trip_ends        num_trip_starts        num_trips  \
route_type                      2      3               2      3         2   
datetime                                                                    
2014-05-26 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-26 12:00:00          39.0  364.0            36.0  343.0      40.0   
2014-05-27 00:00:00           NaN    NaN             NaN    NaN       NaN   
2014-05-27 12:00:00           NaN    NaN             NaN    NaN       NaN   
2014-05-28 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-28 12:00:00          39.0  364.0            36.0  343.0      40.0   

indicator                  service_distance              service_duration  \
route_type               3                2            3                2   
datetime                                                                    
2014-05-26 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-26 12:00:00  368.0      1216.570247  7537.049652        36.333333   
2014-05-27 00:00:00    NaN              NaN          NaN              NaN   
2014-05-27 12:00:00    NaN              NaN          NaN              NaN   
2014-05-28 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-28 12:00:00  368.0      1216.570247  7537.049652        36.333333   

indicator                       service_speed             
route_type                    3             2          3  
datetime                                                  
2014-05-26 00:00:00  154.816667     32.243592  28.050950  
2014-05-26 12:00:00  260.433333     33.483585  28.940418  
2014-05-27 00:00:00         NaN           NaN        NaN  
2014-05-27 12:00:00         NaN           NaN        NaN  
2014-05-28 00:00:00  154.816667     32.243592  28.050950  
2014-05-28 12:00:00  260.433333     33.483585  28.940418

In [103]:
t = pd.DataFrame({"a": [0, np.nan], "b": [0, np.nan]})
t["c"] = (t.a/t.b).fillna(t.a)
t

,a,b,c
0,0.0,0.0,0.0
1,NaN,NaN,NaN


In [82]:
fts.index

DatetimeIndex(['2014-05-26 00:00:00', '2014-05-26 12:00:00',
               '2014-05-27 00:00:00', '2014-05-27 12:00:00',
               '2014-05-28 00:00:00', '2014-05-28 12:00:00'],
              dtype='datetime64[ns]', name='datetime', freq='12H')

In [74]:
num_dates = len(set(t.index.date))
t = (
    gt.unstack_time_series(rts)
    .merge(feed.routes.filter(["route_id", "route_type"]), how="left")
    .groupby(["datetime", "indicator", "route_type"])
    .agg({"value": lambda x: x.sum(skipna=False)})  # Preserve NaNs
    .reset_index("datetime")
    .pivot_table(index=["datetime"], columns=["indicator", "route_type"], aggfunc=lambda x: x.sum(skipna=False))
).value
if num_dates > 1:
    # Insert missing dates and NaNs to complete series index
    end_datetime = pd.to_datetime(f"{t.index.date[-1]:%Y-%m-%d}" + " 23:59:59")
    new_index = pd.date_range(t.index[0], end_datetime, freq=freq, name="datetime")
    t = t.reindex(new_index)

t

indicator           num_trip_ends        num_trip_starts        num_trips  \
route_type                      2      3               2      3         2   
datetime                                                                    
2014-05-26 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-26 12:00:00          39.0  364.0            36.0  343.0      40.0   
2014-05-27 00:00:00           NaN    NaN             NaN    NaN       NaN   
2014-05-27 12:00:00           NaN    NaN             NaN    NaN       NaN   
2014-05-28 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-28 12:00:00          39.0  364.0            36.0  343.0      40.0   

indicator                  service_distance              service_duration  \
route_type               3                2            3                2   
datetime                                                                    
2014-05-26 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-26 12:00:00  368.0      1216.570247  7537.049652        36.333333   
2014-05-27 00:00:00    NaN              NaN          NaN              NaN   
2014-05-27 12:00:00    NaN              NaN          NaN              NaN   
2014-05-28 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-28 12:00:00  368.0      1216.570247  7537.049652        36.333333   

indicator                       service_speed              
route_type                    3             2           3  
datetime                                                   
2014-05-26 00:00:00  154.816667     32.243592  431.515467  
2014-05-26 12:00:00  260.433333     33.483585  575.077777  
2014-05-27 00:00:00         NaN           NaN         NaN  
2014-05-27 12:00:00         NaN           NaN         NaN  
2014-05-28 00:00:00  154.816667     32.243592  431.515467  
2014-05-28 12:00:00  260.433333     33.483585  575.077777

In [59]:
t.

AttributeError: 'numpy.ndarray' object has no attribute 'drop_duplicates'

2

In [39]:
# Slicing 
display(fts.loc[:, ("num_trips", slice(None))])
display(fts.xs("num_trips", axis="columns"))

indicator           num_trips       
route_type                  2      3
datetime                            
2014-05-26 00:00:00      23.0  219.0
2014-05-26 12:00:00      40.0  368.0
2014-05-27 00:00:00      23.0  219.0
2014-05-27 12:00:00      40.0  368.0

route_type,2,3
datetime,,
2014-05-26 00:00:00,23.0,219.0
2014-05-26 12:00:00,40.0,368.0
2014-05-27 00:00:00,23.0,219.0
2014-05-27 12:00:00,40.0,368.0


In [25]:
f = fts.copy()
display(f)
display(gt.unstack_time_series(f))
f.pipe(gt.unstack_time_series).pipe(gt.restack_time_series)



indicator           num_trip_ends        num_trip_starts        num_trips  \
route_type                      2      3               2      3         2   
datetime                                                                    
2014-05-26 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-26 12:00:00          39.0  364.0            36.0  343.0      40.0   
2014-05-27 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-27 12:00:00          39.0  364.0            36.0  343.0      40.0   

indicator                  service_distance              service_duration  \
route_type               3                2            3                2   
datetime                                                                    
2014-05-26 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-26 12:00:00  368.0      1216.570247  7537.049652        36.333333   
2014-05-27 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-27 12:00:00  368.0      1216.570247  7537.049652        36.333333   

indicator                       service_speed             
route_type                    3             2          3  
datetime                                                  
2014-05-26 00:00:00  154.816667     32.243592  28.050950  
2014-05-26 12:00:00  260.433333     33.483585  28.940418  
2014-05-27 00:00:00  154.816667     32.243592  28.050950  
2014-05-27 12:00:00  260.433333     33.483585  28.940418

,datetime,indicator,route_type,value
0,2014-05-26 00:00:00,num_trip_ends,2,19.000000
4,2014-05-26 00:00:00,num_trip_ends,3,195.000000
8,2014-05-26 00:00:00,num_trip_starts,2,23.000000
12,2014-05-26 00:00:00,num_trip_starts,3,220.000000
16,2014-05-26 00:00:00,num_trips,2,23.000000
20,2014-05-26 00:00:00,num_trips,3,219.000000
24,2014-05-26 00:00:00,service_distance,2,677.652816
28,2014-05-26 00:00:00,service_distance,3,4342.754519
32,2014-05-26 00:00:00,service_duration,2,21.016667
36,2014-05-26 00:00:00,service_duration,3,154.816667


indicator           num_trip_ends        num_trip_starts        num_trips  \
route_type                      2      3               2      3         2   
datetime                                                                    
2014-05-26 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-26 12:00:00          39.0  364.0            36.0  343.0      40.0   
2014-05-27 00:00:00          19.0  195.0            23.0  220.0      23.0   
2014-05-27 12:00:00          39.0  364.0            36.0  343.0      40.0   

indicator                  service_distance              service_duration  \
route_type               3                2            3                2   
datetime                                                                    
2014-05-26 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-26 12:00:00  368.0      1216.570247  7537.049652        36.333333   
2014-05-27 00:00:00  219.0       677.652816  4342.754519        21.016667   
2014-05-27 12:00:00  368.0      1216.570247  7537.049652        36.333333   

indicator                       service_speed             
route_type                    3             2          3  
datetime                                                  
2014-05-26 00:00:00  154.816667     32.243592  28.050950  
2014-05-26 12:00:00  260.433333     33.483585  28.940418  
2014-05-27 00:00:00  154.816667     32.243592  28.050950  
2014-05-27 12:00:00  260.433333     33.483585  28.940418

In [ ]:
feed1 = feed.copy()
cal = feed1.calendar.copy()
cal["monday"] = 0
feed1.calendar = cal
feed1.compute_feed_time_series(trip_stats, dates, freq="12H")

In [ ]:
s1 = pd.Series({"hello": 12, "goodbye": 13})
s2 = s1.copy()
pd.DataFrame([s1, s2])

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()